#Продвинутый Python, ДЗ-4

Правила игры:

В домашке 6 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно рисовать графики с помощью plotly или же Dash. Обратите внимание, что все графики и оси должны быть подписаны, графики должны выглядеть аккуратно (за это будем штрафовать)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=d8fd7e3430ec6391975caf848b97be0629fb3a465124d8abe4057c11cbfe9c31
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 68.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget
import plotly
import plotly.graph_objs as go
import plotly.express as px
from dateutil.parser import parse
import datetime
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

<ipython-input-3-0aeea3e0a49f>:13: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-0aeea3e0a49f>:14: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

# Часть 1. Plotly

В данной части для заданий нужно использовать Plotly для того, чтобы отобразить графики

## Задание 1 (15 баллов)

Отобразите на карте всех продавцов (в качестве точек - координаты, в качестве размера и цвета точек - число продавцов). При наведении точек должно отображаться название штата, города, а также число продавцов

*Hint: внимательно посмотрите на датасет locations. Если делать задачу максимально втупую, то у вас либо крашнется/зависнет отрисовка, либо на это нельзя будет нормально смотреть*

In [4]:
sel_cities = sellers.groupby(['seller_city', 'seller_state'], as_index=False).agg({'seller_id': 'count'})
loc_cities = location.groupby(['geolocation_city', 'geolocation_state'], as_index=False).agg({'geolocation_lat': 'mean', 'geolocation_lng': 'mean'})
loc_cities.rename(columns = {'geolocation_city': 'seller_city', 'geolocation_state': 'seller_state'}, inplace=True)
sel_cities = sel_cities.merge(loc_cities, on=['seller_city', 'seller_state'], how='left')

In [5]:
def to_int_size(value):
    return 2*np.log2(int(value))


sel_cities['log_sel_cnt'] = sel_cities['seller_id'].map(to_int_size)

fig = go.Figure(go.Scattermapbox(lat=sel_cities['geolocation_lat'],
                                 lon=sel_cities['geolocation_lng'],
                                 text=sel_cities[['seller_city', 'seller_state', 'seller_id']],
                                 marker=dict(colorbar=dict(title="Число продавцов"),
                                             color=sel_cities['log_sel_cnt'],
                                             size=sel_cities['log_sel_cnt'])))

fig.update_layout(
    title_text = 'Продавцы на карте'
)


cap = sel_cities[sel_cities['seller_city']=='alambari']
map_center = go.layout.mapbox.Center(lat=cap['geolocation_lat'].values[0], lon=cap['geolocation_lng'].values[0])
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=4))
fig.show()


## Задание 2 (15 баллов)

Отобразите на графике число заказов по дням с разбивкой по статусам. Также создайте график со средней оценкой отзыва (аггрегируйте по дням). Для обоих графиков должен быть слайдер по датам

In [6]:
orders['order_purchase_date'] = orders['order_purchase_timestamp'].apply(lambda x: parse(x, yearfirst=True).strftime('%Y-%m-%d'))

In [7]:
fig = px.histogram(
    orders,
    x = 'order_purchase_date',
    color = 'order_status',
    barmode = 'stack'
)

fig.update_layout(
    height=900,
    title_text = 'Число заказов по дням',
    xaxis_title_text = 'Дата',
    yaxis_title_text = 'Число заказов',
    bargap = 0.2
)

fig.update_xaxes(
    type='category',
    categoryorder='category ascending',
    title='Дата'
)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        )
    )
)

fig.show()

In [8]:
ord_review = orders.merge(reviews, on='order_id', how='left')
ord_review = ord_review.groupby('order_purchase_date', as_index=False).agg({'review_score': 'mean'})


fig = px.bar(ord_review,
                   x = 'order_purchase_date',
                   y = 'review_score'
                   )

fig.update_layout(
    height=900,
    title_text = 'Средняя оценка отзывов по дням',
    xaxis_title_text = 'Дата',
    yaxis_title_text = 'Средняя оценка',
    bargap = 0.2
)

fig.update_xaxes(
    type='category',
    categoryorder='category ascending',
    title='Дата'
)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        )
    )
)

fig.show()

## Задание 3 (20 баллов)

Выберите топ-5 продавцов с более чем 20 продажами, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

In [9]:
res = items.merge(orders, on='order_id', how='left')
res = res.merge(customers, on='customer_id', how='left')
res = res.merge(sellers, on='seller_id', how='left')
# res = res[res['order_status'] == 'delivered']

res['another_state'] = (res.customer_state != res.seller_state)
res = res[['seller_id', 'order_id', 'another_state', 'customer_state', 'seller_state', 'customer_city', 'seller_city']]
res = res.drop_duplicates()

res_gr = res.groupby(['seller_id'], as_index=False).agg({'another_state' : 'sum', 'order_id' : 'nunique'})
res_gr = res_gr[res_gr.order_id > 20]
res_gr['share'] = res_gr.another_state / res_gr.order_id
res_gr.sort_values(by ='share', ascending=False, inplace=True)
sellers_top5 = set(res_gr[:5].seller_id)
res = res[res['seller_id'].isin(sellers_top5)]

loc_cities = location.groupby(['geolocation_city', 'geolocation_state'], as_index=False).agg({'geolocation_lat': 'mean', 'geolocation_lng': 'mean'})
res = res.merge(loc_cities, left_on=['customer_city', 'customer_state'], right_on=['geolocation_city', 'geolocation_state'], how='left')
res = res.rename(columns={'geolocation_lat': 'cust_lat', 'geolocation_lng': 'cust_lng'})
res = res.merge(loc_cities, left_on=['seller_city', 'seller_state'], right_on=['geolocation_city', 'geolocation_state'], how='left')
res = res.rename(columns={'geolocation_lat': 'sel_lat', 'geolocation_lng': 'sel_lng'})
res = res[['seller_id', 'cust_lat', 'cust_lng', 'sel_lat', 'sel_lng', 'order_id']]

fig = go.Figure()


fig.update_layout(
    title_text = 'Посылки топ-5 продавцов с более чем 20 продажами, которые чаще всего отсылают свою посылку в другие регионы'
)

for index, row in res.iterrows():
    fig.add_trace(go.Scattermapbox(
        mode='lines',
        lon=[row.sel_lng, row.cust_lng],
        lat=[row.sel_lat, row.cust_lat],
        line=dict(width=2),
        marker=dict(
            size=10,
        ),
        text=row['order_id']
    ))

    fig.data[-1].update(showlegend=False)


cap = sel_cities[sel_cities['seller_city']=='sao paulo']
map_center = go.layout.mapbox.Center(lat=cap['geolocation_lat'].values[0], lon=cap['geolocation_lng'].values[0])
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=3))
fig.show()

# Часть 2. Dash

В данной части нужно сделать и сдать один .py файл (можно назвать dash_task.py), который будет разворачивать страничку с нарисованными графиками (с помощью Dash)

## Задание 4 (20 баллов)

Сделайте сайт, где будет отображаться следующее:

* График, показывающий распределение по категориям продаж (то есть по тому, что продают селлеры) в штате

* График, показывающий распределение по категориям покупок (то есть по тому, что покупают пользователи) в штате

* Фильтр по штатам (должна быть возможность как выбрать отдельный штат, так и вся Бразилия в целом)

* Возможность выбрать, какие статусы продаж учитывать (чтобы можно было выбрать несколько статусов)

* Слайдер по датам (чтобы можно было выбрать даты, по которым можем смотреть результаты)


## Задание 5 (15 баллов)

Добавить карту, на котором можно будет посмотреть на количество продавцов или на количество покупателей (на него также должны работать фильтры по датам и по статусам, реализованные в прошлом занятии). Аггрегацию нужно делать по штатам. Чтобы вам помочь, вот snippet кода, который можно использовать для отображения:

In [10]:
import json
import pandas as pd
import plotly as plt
import plotly.express as px
from urllib.request import urlopen
import json

# Brazil coordinates / shape
with urlopen(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
) as response:
    brazil = json.load(response)

all_states = set()

# Since the database doesn't have an ID or feature using which values will be mapped between the coordinate/shape database and soybean database, we are adding an ID ourselves.
for feature in brazil["features"]:
    feature["id"] = feature["properties"]["sigla"]
    all_states.add(feature["id"])


*Hint: обратите внимание, что покупатели и продавцы могут быть не во всех штатах, а эжто значит, что они могут не отображаться на карте. Эту ситуацию нужно обрабатывать и поправить*

## Задание 6 (15 баллов)

Сделайте карту кликабельной: по клику на штат должен показываться сам штат (и только он), все остальные графики обновляются по выбранному штату